### Testing New csv Load In Method

In [49]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import theano
import spacy
import s3
nlp = spacy.load('en')

In [50]:
# Everytime you enter
bucket = s3.Bucket('yelpchallenge1')

In [51]:
bucket.contents

['datasets/',
 'datasets/df.csv',
 'notebooks/',
 'notebooks/data_cleanup.ipynb',
 'notebooks/vectorization_exploration.ipynb',
 'notebooks/yelp_data_initial_exploration.ipynb']

In [47]:
# Only have to run this once. 
#bucket.get('datasets/df.csv', 'df.csv')

In [52]:
# import final .csv
df = pd.read_csv('df.csv')

In [40]:
# check import
df.head()

,stars,text,date,total_votes,tokens
0,1,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,7.0,"['total', 'horrible', 'service', 'crooks', 'ac..."
1,5,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,0.0,"['adore', 'travis', 'hard', 'rock', 'kelly', '..."
2,5,I have to say that this office really has it t...,2016-11-09 20:09:03,3.0,"['office', 'organized', 'friendly', 'phillipp'..."
3,5,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,0.0,"['went', 'lunch', 'steak', 'sandwich', 'delici..."
4,1,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,7.0,"['today', 'second', 'sessions', 'paid', 'sessi..."


In [53]:
# check datatypes of columns in dataframe
df.dtypes

Unnamed: 0      object
stars           object
text            object
date            object
total_votes    float64
tokens          object
dtype: object

In [54]:
# drop unwanted columns
df = df.drop(columns=['Unnamed: 0'])

In [55]:
df.head()

,stars,text,date,total_votes,tokens
0,1,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,7.0,"['total', 'horrible', 'service', 'crooks', 'ac..."
1,5,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,0.0,"['adore', 'travis', 'hard', 'rock', 'kelly', '..."
2,5,I have to say that this office really has it t...,2016-11-09 20:09:03,3.0,"['office', 'organized', 'friendly', 'phillipp'..."
3,5,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,0.0,"['went', 'lunch', 'steak', 'sandwich', 'delici..."
4,1,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,7.0,"['today', 'second', 'sessions', 'paid', 'sessi..."


In [56]:
df.dtypes

stars           object
text            object
date            object
total_votes    float64
tokens          object
dtype: object

In [57]:
# create mini-dataframe for testing
# want to make sure works locally on small dataset before scaling to entire dataset/AWS
#mini_df = df.head(10)

In [58]:
# exploring TFIDF Vectorizer

In [59]:
# new tokenizer
def tokenize(document):
    doc = nlp(document)
    return [token.lemma_ for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [60]:
# create variable to feed into TFIDF Vectorizer fit_transform
# to be updated to 'text' column of main dataframe (df['text']) for vectorization in AWS
data = df['text']

In [61]:
data[2]

"I have to say that this office really has it together, they are so organized and friendly!  Dr. J. Phillipp is a great dentist, very friendly and professional.  The dental assistants that helped in my procedure were amazing, Jewel and Bailey helped me to feel comfortable!  I don't have dental insurance, but they have this insurance through their office you can purchase for $80 something a year and this gave me 25% off all of my dental work, plus they helped me get signed up for care credit which I knew nothing about before this visit!  I highly recommend this office for the nice synergy the whole office has!"

In [ ]:
# vectorizer
tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))

# Create a vocabulary and get word counts per document
# Learn vocab and transform data into form we want
sparse = tfidf.fit_transform(data)

# Print word counts

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
#dtm.head(50)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
# View Feature Matrix as DataFrame
dtm.head(50)